<a href="https://colab.research.google.com/github/Unisub23/python-project/blob/main/Expense_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from datetime import datetime
from typing import Dict
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# ----------------------------
# ExpenseTracker (storage, summary, plotting)
# ----------------------------
class ExpenseTracker:
    def __init__(self, filename: str = "expenses.csv"):
        self.filename = filename
        if not os.path.exists(self.filename):
            df = pd.DataFrame(columns=["Date", "Name", "Amount", "Category"])
            df.to_csv(self.filename, index=False)

    def _ensure_numeric(self, value):
        try:
            return float(value)
        except Exception:
            raise ValueError("Amount must be a number (e.g., 150.50)")

    def add_expense(self, name: str, amount, category: str) -> Dict:
        name = str(name).strip()
        amount = self._ensure_numeric(amount)

        if not name:
            raise ValueError("Expense name cannot be empty.")
        if amount <= 0:
            raise ValueError("Amount must be greater than zero.")
        if not category:
            raise ValueError("Please choose a category.")

        date = datetime.now().strftime("%Y-%m-%d")
        new_row = pd.DataFrame([[date, name, amount, category]],
                               columns=["Date", "Name", "Amount", "Category"])
        new_row.to_csv(self.filename, mode="a", header=False, index=False)
        return {"Date": date, "Name": name, "Amount": amount, "Category": category}

    def get_data(self) -> pd.DataFrame:
        return pd.read_csv(self.filename)

    def get_summary_df(self) -> pd.DataFrame:
        df = self.get_data()
        if df.empty:
            return pd.DataFrame(columns=["Category", "Total (₹)"])
        summary = df.groupby("Category", as_index=False)["Amount"].sum().rename(columns={"Amount": "Total (₹)"})
        # sort descending for nicer display
        summary = summary.sort_values(by="Total (₹)", ascending=False).reset_index(drop=True)
        return summary

    def plot_summary(self, figsize=(10, 6)) -> plt.Figure:
        summary = self.get_summary_df()
        if summary.empty or summary["Total (₹)"].sum() == 0:
            raise ValueError("No expenses to plot.")
        fig, ax = plt.subplots(figsize=figsize)
        bars = ax.bar(summary["Category"], summary["Total (₹)"])
        ax.set_title("Total Expenses by Category", fontsize=14, fontweight="bold")
        ax.set_xlabel("Category", fontsize=12)
        ax.set_ylabel("Amount (₹)", fontsize=12)
        ax.set_xticks(range(len(summary["Category"])))
        ax.set_xticklabels(summary["Category"], rotation=45, ha="right")
        ax.bar_label(bars, labels=[f"₹{v:,.2f}" for v in summary["Total (₹)"]], padding=3, fontsize=9)
        plt.tight_layout()
        return fig

    def export(self, out_file="exported_expenses.csv") -> str:
        df = self.get_data()
        df.to_csv(out_file, index=False)
        return out_file

# ----------------------------
# UI (ipywidgets)
# ----------------------------
expense_categories = [
    "🍕 Food",
    "✈️ Travel",
    "🎉 Fun",
    "📚 Study",
    "💼 Work",
    "📦 Miscellaneous",
    "💰 Saving"
]

tracker = ExpenseTracker()

# Inputs
name_input = widgets.Text(placeholder="e.g. Lunch at cafe", description="Name:")
amount_input = widgets.BoundedFloatText(value=0.0, min=0.0, max=1e9, step=0.5, description="Amount ₹:")
category_dropdown = widgets.Dropdown(options=expense_categories, description="Category:")
add_btn = widgets.Button(description="Add Expense", button_style="success", tooltip="Add expense")
summary_btn = widgets.Button(description="Show Summary", button_style="info")
plot_btn = widgets.Button(description="Show Graph", button_style="warning")
export_btn = widgets.Button(description="Export & Download", button_style="primary")
output = widgets.Output(layout={'border': '1px solid lightgray'})

# Helper that clears widget output area and runs an action
def run_in_output(action):
    output.clear_output(wait=True)
    with output:
        try:
            action()
        except Exception as e:
            print("⚠️", e)

# Button callbacks
def on_add_clicked(_):
    def action():
        res = tracker.add_expense(name_input.value, amount_input.value, category_dropdown.value)
        print(f"✅ Added: {res['Name']} | ₹{res['Amount']:.2f} | {res['Category']} | {res['Date']}")
        # reset inputs
        name_input.value = ""
        amount_input.value = 0.0
    run_in_output(action)

def on_summary_clicked(_):
    def action():
        df = tracker.get_summary_df()
        if df.empty:
            print("⚠️ No expenses recorded yet.")
        else:
            print("📊 Expense Summary (by Category):")
            display(df.style.format({"Total (₹)": "₹{:.2f}"}))
    run_in_output(action)

def on_plot_clicked(_):
    def action():
        fig = tracker.plot_summary()
        display(fig)
        plt.close(fig)
    run_in_output(action)

def on_export_clicked(_):
    def action():
        out_file = tracker.export()
        print(f"📂 Exported to `{out_file}`")
        # Attempt to trigger download if running in Colab
        try:
            from google.colab import files
            files.download(out_file)
        except Exception:
            print("Tip: If running in Colab, the file should download automatically. Otherwise find the file in the notebook's working directory.")
    run_in_output(action)

# Wire buttons
add_btn.on_click(on_add_clicked)
summary_btn.on_click(on_summary_clicked)
plot_btn.on_click(on_plot_clicked)
export_btn.on_click(on_export_clicked)

# Layout & display
controls = widgets.VBox([
    widgets.HTML("<h3>💼 Professional Expense Tracker (Colab)</h3>"),
    name_input,
    amount_input,
    category_dropdown,
    widgets.HBox([add_btn, summary_btn, plot_btn, export_btn]),
    widgets.HTML("<small>Notes: All data is saved to <code>expenses.csv</code> in the notebook environment.</small>"),
])

display(widgets.HBox([controls, output], layout=widgets.Layout(gap="16px")))
